# Cálculo score Support
  
En el presente archivo se realizará la comparativa entre jugadores profesionales y los jugadores que tienen potencial para serlo.
  
En este documento se sigue exactamente el mismo proceso llevado acabo en el archivo 'jun_clasificacion.ipynb'. Se recomienda su consulta para el pleno entendimiento del mismo.

In [1]:
import numpy as np
import pandas as pd
import time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder

from scipy.spatial.distance import pdist, squareform

In [2]:
rookies = pd.read_csv('../data/total.csv')
profesional = pd.read_csv('../data/total_prof.csv')

In [3]:
prof = profesional[['nombre_invocador', 'lps', 'ranking_mundial', 'ranking_porcentaje', 'wins', 'loses', 'n_games', 'winrate_total', 'liga_label', 'kills_player_kda', 'deaths_player_kda', 'assists_player_kda', 'rol_1', 'games_per_rol_1', 'winrate_per_rol_1', 'champ_1', 'regional_rank_1', 'kills_champ_1', 'deaths_champ_1', 'assists_champ_1', 'games_champ_1', 'winrate_champ_1', 'champ_2', 'regional_rank_2', 'kills_champ_2', 'deaths_champ_2', 'assists_champ_2', 'games_champ_2', 'winrate_champ_2', 'champ_3', 'regional_rank_3', 'kills_champ_3', 'deaths_champ_3', 'assists_champ_3', 'games_champ_3', 'winrate_champ_3', 'champ_4', 'regional_rank_4', 'kills_champ_4', 'deaths_champ_4', 'assists_champ_4', 'games_champ_4', 'winrate_champ_4', 'nivel']]

rook = rookies[['nombre_invocador', 'lps', 'ranking_mundial', 'ranking_porcentaje', 'wins', 'loses', 'n_games', 'winrate_total', 'liga_label', 'kills_player_kda', 'deaths_player_kda', 'assists_player_kda', 'rol_1', 'games_per_rol_1', 'winrate_per_rol_1', 'champ_1', 'regional_rank_1', 'kills_champ_1', 'deaths_champ_1', 'assists_champ_1', 'games_champ_1', 'winrate_champ_1', 'champ_2', 'regional_rank_2', 'kills_champ_2', 'deaths_champ_2', 'assists_champ_2', 'games_champ_2', 'winrate_champ_2', 'champ_3', 'regional_rank_3', 'kills_champ_3', 'deaths_champ_3', 'assists_champ_3', 'games_champ_3', 'winrate_champ_3', 'champ_4', 'regional_rank_4', 'kills_champ_4', 'deaths_champ_4', 'assists_champ_4', 'games_champ_4', 'winrate_champ_4', 'nivel']]

In [4]:
rook = rook[~rook['nombre_invocador'].isin(prof['nombre_invocador'].unique())]

In [5]:
data = pd.concat([rook, prof])

In [6]:
data = data.fillna('Unknown')

In [7]:
le=LabelEncoder()

for c in data.columns:
    if c != 'nombre_invocador' and c != 'rol_1' and c != 'nivel':
        if data.dtypes[c]==object:

            le.fit(data[c].astype(str))

            data[c]=le.transform(data[c].astype(str))

In [8]:
supp = data.loc[(data.rol_1 == 'Support')]

In [9]:
supp = supp.drop_duplicates()

In [10]:
supp.nivel = supp.nivel.apply(lambda x: 1 if x == 'profesional' else 0)

In [11]:
supp = supp.set_index('nombre_invocador')

In [12]:
supp.drop('rol_1', axis = 1, inplace = True)

In [13]:
supp.head()

,lps,ranking_mundial,ranking_porcentaje,wins,loses,n_games,winrate_total,liga_label,kills_player_kda,deaths_player_kda,assists_player_kda,games_per_rol_1,winrate_per_rol_1,champ_1,regional_rank_1,kills_champ_1,deaths_champ_1,assists_champ_1,games_champ_1,winrate_champ_1,champ_2,regional_rank_2,kills_champ_2,deaths_champ_2,assists_champ_2,games_champ_2,winrate_champ_2,champ_3,regional_rank_3,kills_champ_3,deaths_champ_3,assists_champ_3,games_champ_3,winrate_champ_3,champ_4,regional_rank_4,kills_champ_4,deaths_champ_4,assists_champ_4,games_champ_4,winrate_champ_4,nivel
nombre_invocador,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
frosty,1760,3,0.000032,194,141,336,57.7,2,5.1,5.0,12.4,249,57.4,9,1,3.6,4.9,13.5,68,52.9,53,21,4.2,4.5,16.8,34,64.7,41,70,4.6,5.4,11.6,34,58.8,95,37,2.0,4.1,17.4,33,60.6,0
scamber,929,634,0.006800,138,122,260,53.1,2,2.6,5.9,14.5,255,53.7,9,173,3.1,7.5,14.4,50,44.0,40,77,3.2,6.3,10.6,24,54.2,82,161,2.0,5.5,14.0,23,52.2,54,277,2.3,5.4,16.9,18,50.0,0
Skýlinè,845,951,0.010000,224,193,418,53.8,2,5.6,6.6,8.2,222,47.3,88,284,4.7,8.1,8.9,183,48.1,144,45,7.1,4.6,6.9,171,61.4,93,17,3.5,9.3,9.5,13,46.2,61,3814,7.6,4.6,4.3,7,42.9,0
zlaksd,923,657,0.007000,137,105,248,56.5,2,2.2,4.2,11.6,220,59.5,52,158,2.5,3.7,12.3,75,61.3,95,22,1.1,4.3,12.3,27,51.9,82,1226,0.9,5.4,14.7,21,71.4,127,405,1.8,4.4,14.3,11,81.8,0
INT Baul,845,956,0.010000,87,55,165,58.8,2,3.5,4.8,12.9,149,61.1,9,289,3.2,4.8,15.1,14,71.4,66,3927,1.8,5.8,14.5,13,61.5,82,1682,2.0,4.2,12.9,10,90.0,80,2155,4.1,4.7,16.3,10,50.0,0


In [14]:
distancias = squareform(pdist(supp, 'euclidean'))    

distancias.shape

(597, 597)

In [15]:
similitud = 1 / (1 + distancias)

similitud.shape

(597, 597)

In [16]:
supp_sim = pd.DataFrame(similitud, index=supp.index, columns=supp.index)

supp_sim.head()

nombre_invocador,frosty,scamber,Skýlinè,zlaksd,INT Baul,MaioLex,Twitchtv Crecre,BioPanther,LOUD Ceos,Bίlly Butcher,ACKÈRMAN,M1nit,Kïer,pocovirtuoso,DK Bible,coldxn,ProDelta,Pagorex,Vitu 9,donbray,Armored Mawdad,4321455213123,beansu,SnowFlow2r,999 1v9 L9 69 v9,DeadVinland,Kasing 1,Thumbs Down,bountyyspider,Bleecky,SALAH 11,Isles1,sup momochi,INTZ Nia,FX Jojinho,kStars Telas,Path To LCS,et pas une autre,Feliaaa,Swkeeee,HenriSthel,Elake,СВЕКЛОВИЧНЫЙ ЖОМ,BIade Runner2049,Gmandlife,Mixtsure,Echte Liebe,Procurando Nami,Chu성훈,ψτροφήψ,JKG Qu1z,printer break,Cl0x2227,aRwinGü,BOC tufi,Liberty Momochi,RNG Ming 9,MURILAAAAAAAAAAO,scuro,Benvi,사람 안 믿어,Jan D Banan,Cosboat,Donna Hamilton,messi sup acc,winter,Down Kim,QQ LOVE,vaggut,DK Loopy,Just an engineer,picapicaa,shibidi bob yes,Harunohi,KT Way,pienso,Noa love Brazil,05222001,KDF Andil,Que bendición,100 Busio,thominhas,다이아몬드,FA Zodiac,imagine a legend,Dam ADC,Calmsky,MarceSato,Lekcyc,P1ng,AMT,Renata Enjoyer,Momo3,zenbokorosn,Zyors,lisan hehe,PinkMin v2,Advienne,LooserQ Member,xooos,Shoiti,nyshotytskazat,Nubobo,ayekasia,bddst shp,V Julius Caesar,YoojungHukiriLee,Pockus,LANZIQUEEN,twtv rocks908,Nmcp,Messages Sup Acc,dtro1,小西寶寶A,donkey gap,Kick a boo,kylie jenner,kevinnguyensocal,never one,younce,TSM Chime,Garrett Meyerson,amphy,You,boba dopa,Smoothie,Colchønsito,uZent,ttv donbraylol,RIV kshhh,ReturnOfTheΚing,Try again later,vvx1585410146,raw outlaw,C9 Zven,SharkNad0,paiN Damage1,Louvi,El Camino provee,Riiquee,FUR Manel,Airobloodin,Airobloodin,RISE Wosz,NORDKASING,슈퍼히어로6,TTV White Lotus,la mas envidiosa,Breezyyy,Rascal,Douzheng,Lugarit,Hantera,weixin7777777,Baleada,Diviner,กระเบน,我們可是純愛呢,GSE Azazel,Sätori,융 션,DongerValenz,FF Kovu,Amaboutakum,Locura y MaIdad,PCG PhantomDeath,Talentoso,Talentoso,sxtxt supp acc,Adrayon,Gakseong,tuna6666,Z Score,Arecent,maestro tomberi,maestro tomberi,Davion,Gen G Namgung,Loggan Double G,Zyko,Pandorá,Zlatorzova cola,ACP 5231,Son Günüme Kadar,SHOP MRBEAST,wustebulgarski,NvldR,ƒïnn,SEN Ezmoou,Dejmianov,Pyku,Hi Hyung,舒華寶,nezhilil noah7,TTV Jealowstar,dfm1,IIIlIlIlIlIlIII,Bengali Tiger,Camilo,Ronjac,Ahriana Grande,AMG,SmashSCY,THE GIEF,14675242,藍雙布偶,jiazzzzz,vりんごちゃんv,PHTOM,T1nyyy,DRKSHDW BY RO,LLP Gaya,Shingeki No Poro,칸쵸빌런,Thresh Maker,Thresh Maker,NoisyLand,Hero arc,bae lú lú,Parlak Pasif Gay,Dammit Rakan,Farketmez1256,Farketmez1256,13another,love engage,Señorita,Pisces Babe,Morgenstern,BLG 온,グレイラースＺＺ,enpad,MKL Sol,Giselle,Sunn Mond,Nebulim,高振哲,Audire,PDT AskaMoon,유징쓰,유징쓰,Shaozhe1,Amor eres tu,Ryejin,xrzs,Chcknz,Adela Andromeda,thots,수요일은다먹는날,스트레스로돌연사,스트레스로돌연사,pcxcxq,Doveee,Pa k juego esto,Trymmo,Trymmo,배빵빵 재훈이,TSK Milfhunter,va mot lan cuoi,vvxinRio887288,百鬼丸,1Zheyss,1Pasha,Hououín Kyouma,EXPERT VELKOZ,TTV Billyjimi,暖 冬,do better,SNDR,선제공격 카르마,Q vsj,Usually Gapping,摯友維尼,sweetie today,ca phe muoi,RLPKVNCVLLR,ttvElcariwea,Αshura,callate boludito,Chebi,Omiss,la cariñosaa,WindüChaser,Cato Mart,CaryKung,UNSECRET,Bchau,ycx,Attenrislip,Its me ZEON,Narcissisticc,dampiro98,xxiixxiixx,鵝鵝小騎士,Lire,Yuukiyuuna,TF TF,HEUN XINH GÁI,DuDu111,High Mental,Brinley,nataliuauaua,Vaco,ΒΑTMAN,PI Blade40,ClickJJ,Tarmi,Eskiper,morale support,Enchanters xd,winplease123,Kongo TV Dzicz,DKB Sinnlos,Pinguím,40yo glinomes,jüninho,Veignorem,Aesenar,Caüse,Nujabes Luvsic,DDDORIAN,RLB Brother Luis,VOICES ARE ßACK,Marilamilk,미니웅즈,SIGMA MENTALlTY,RED Stoneses,UpDMP,Giguiron,PsYcraw,MILO J,solukmavinokta,Nag7,69Chaz420,3uphoria,4th June,LickMyHeaI,Rhÿ,megx ª,S0 Am i,Tenaka Khan,Juppiterx,FNTSY Pandá,Aikawa Kizuna,ehxc9,Road2Narnia,im a cυte girl,Babygirl Laura,zimba int acc,SCCP joseph,t d,Unméi,flesh n blood,hyne chan,AST Rylle,Ulí,Carrry,Y4ma,batman vermelho,Tiago Rodrigues,Hiccup1357,POLITICAGEM,MWD Kidiro v9,Opção Número 2,2knarkearb,Redentor,Wekin Poof,LNG Hang,Matsouza,Paolocannone,keriamingberyl1,braydon,Linkz,Prymari,Avuhlie,its Barlo,its Barlo,MOO5257,rwta th mana s,Protos,Ameericle,ofnir,TOURO REPRODÜTOR,5word,LiL TommyG,COMENDO LIXO,COMENDO LIXO,crawling soul,eustachian,pure empathy,Abu Andr

In [17]:
prof.nombre_invocador.loc[prof.rol_1 == 'Support']

5       FNC Hylissang
15    TL Honda CoreJJ
16          Vulcan 01
17                역천괴
22      school phobia
23           mersa777
26           Targamas
31           KC Leona
34       Zaunite Mylo
Name: nombre_invocador, dtype: object

In [18]:
col = supp_sim.columns

In [19]:
col[:-9]

Index(['frosty', 'scamber', 'Skýlinè', 'zlaksd', 'INT Baul', 'MaioLex',
       'Twitchtv Crecre', 'BioPanther', 'LOUD Ceos', 'Bίlly Butcher',
       ...
       'Kita1', 'CuriousJoe', 'CuriousJoe', 'TianXinSup01', 'Sasiki', 'Sasiki',
       'EastGoblin', 'deadprince19', 'deadprince19', '귀여워서 죄송합니다'],
      dtype='object', name='nombre_invocador', length=588)

In [20]:
supp_sim.drop(col[:-9], axis = 1, inplace = True)

In [21]:
supp_sim['Total'] = supp_sim.sum(axis = 1)

In [23]:
supp_mando = supp_sim.sort_values(by = 'Total', ascending = False)

In [24]:
supp_mando.head()

nombre_invocador,FNC Hylissang,TL Honda CoreJJ,Vulcan 01,역천괴,school phobia,mersa777,Targamas,KC Leona,Zaunite Mylo,Total
nombre_invocador,,,,,,,,,,
FNC Hylissang,1.000000,0.000795,0.000365,0.000140,0.000570,0.000224,0.004669,0.000271,0.000049,1.007084
Targamas,0.004669,0.000781,0.000359,0.000137,0.000580,0.000221,1.000000,0.000263,0.000049,1.007059
TL Honda CoreJJ,0.000795,1.000000,0.000522,0.000147,0.001258,0.000260,0.000781,0.000337,0.000050,1.004149
school phobia,0.000570,0.001258,0.000469,0.000138,1.000000,0.000234,0.000580,0.000314,0.000049,1.003612
Vulcan 01,0.000365,0.000522,1.000000,0.000156,0.000469,0.000264,0.000359,0.000350,0.000054,1.002540


In [25]:
supp_mando.to_csv('../data/cuadro_mando/supp_mando.csv')